<a href="https://colab.research.google.com/github/DiegoCaetano/DIO-santander/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Santander Dev Week 2023 (ETL com Python)**

In [ ]:
stn_api_url = 'http://apidatalake.tesouro.gov.br'
ano_referencia = 2023

**Contexto:** Você é um cientista de dados e recebeu a tarefa de identificar os municipos que não entregaram a Matriz de Saldos Contábeis, ou simplesmente MSC ao Site da https://siconfi.tesouro.gov.br/.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de códigos de IBGE do minicipo:
  ```
  IBGE_Id
  2507507
  2500106
  2700102
  
2. Seu trabalho é consumir o endpoint `curl -X 'GET' \
  'https://apidatalake.tesouro.gov.br/ords/siconfi/tt/extrato_entregas?id_ente={IBGE_ID}&an_referencia={ano_referencia}'` (API da Tesouro.gov.br) para obter os dados de cada municipo.

3. Depois de obter os dados dos Municipios, ira verificar qual a instituição esta faltando a entregar a MSC e notifica-la.

4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



## **E**xtract

Extraia a lista de IBGE_Id de municipio a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do municipio correspondente.

In [5]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
municipio_ids = df['IBGE_Id'].tolist()


print(municipio_ids)

[2700102, 2507507, 2500106]


In [6]:
import requests
import json

def get_extrato_entregas(ibge_id, ano_referencia):
  headers = {'Accept': 'application/json'}
  url = f'{stn_api_url}/ords/siconfi/tt/extrato_entregas?id_ente={ibge_id}&an_referencia={ano_referencia}'
  print(url)
  response = requests.get(url, headers=headers,timeout=60)
  return response.json() if response.status_code == 200 else None

extratos_entregas = [entrega for id in municipio_ids if (entrega := get_extrato_entregas(id, ano_referencia)) is not None]
print(json.dumps(extratos_entregas, indent=2))

http://apidatalake.tesouro.gov.br/ords/siconfi/tt/extrato_entregas?id_ente=2700102&an_referencia=2023


ConnectTimeout: ignored

## **T**ransform

Verificando qual a instituição tem alguma pendência de relatório.

In [ ]:
for extrato_entrega in extratos_entregas:
  for item in extrato_entrega["items"]:
    if item['status_relatorio'] == None:
        print(json.dumps(item, indent=2))
